In [1]:
pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow==2.12.0rc0 tensorflow-io==0.29.0 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy pandas scikit-learn tensorflow keras

In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
np.random.seed(0)
# from sklearn.metrics import precision_score,recall_score,f1_score
import tensorflow as tf
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import ConfusionMatrixDisplay
# from sklearn.metrics import confusion_matrix

In [7]:
data = pd.read_csv("Downloads\Project1\labelled_testing_data.csv")
print("Total missing values : ",sum(list(data.isna().sum())))
data


Total missing values :  0


,timestamp,processId,threadId,parentProcessId,userId,mountNamespace,processName,hostName,eventId,eventName,stackAddresses,argsNum,returnValue,args,sus,evil
0,129.050634,382,382,1,101,4026532232,systemd-resolve,ip-10-100-1-217,41,socket,"[140159195621643, 140159192455417, 94656731598...",3,15,"[{'name': 'domain', 'type': 'int', 'value': 'A...",0,0
1,129.051238,379,379,1,100,4026532231,systemd-network,ip-10-100-1-217,41,socket,"[139853228042507, 93935071185801, 93935080775184]",3,15,"[{'name': 'domain', 'type': 'int', 'value': 'A...",0,0
2,129.051434,1,1,0,0,4026531840,systemd,ip-10-100-1-217,1005,security_file_open,"[140362867191588, 8103505641674583858]",4,0,"[{'name': 'pathname', 'type': 'const char*', '...",0,0
3,129.051481,1,1,0,0,4026531840,systemd,ip-10-100-1-217,257,openat,[],4,17,"[{'name': 'dirfd', 'type': 'int', 'value': -10...",0,0
4,129.051522,1,1,0,0,4026531840,systemd,ip-10-100-1-217,5,fstat,[140362867189385],2,0,"[{'name': 'fd', 'type': 'int', 'value': 17}, {...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188962,496.629003,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1
188963,496.629025,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1
188964,496.629047,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1
188965,496.629069,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1


In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

In [30]:
non_numeric_columns = data.select_dtypes(include=['object']).columns
le = LabelEncoder()
for column in non_numeric_columns:
    data[column] = le.fit_transform(data[column])

In [32]:
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for 1D CNN: (samples, time steps, features)
# Here, each sample has a sequence of features (time steps = 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [34]:
model = Sequential()

# Add a 1D Convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# Flatten the output and pass through a Dense layer
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))

# Output layer for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 6, 32)            0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 6, 32)             0         
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 128)               24704     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [36]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 14s 4ms/step - loss: 0.0321 - accuracy: 0.9917 - val_loss: 0.0263 - val_accuracy: 0.9912
Epoch 2/10
3307/3307 [==============================] - 12s 4ms/step - loss: 0.0255 - accuracy: 0.9924 - val_loss: 0.0259 - val_accuracy: 0.9914
Epoch 3/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0246 - accuracy: 0.9924 - val_loss: 0.0244 - val_accuracy: 0.9916
Epoch 4/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0239 - accuracy: 0.9926 - val_loss: 0.0230 - val_accuracy: 0.9914
Epoch 5/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0231 - accuracy: 0.9926 - val_loss: 0.0257 - val_accuracy: 0.9919
Epoch 6/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0230 - accuracy: 0.9926 - val_loss: 0.0229 - val_accuracy: 0.9921
Epoch 7/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0226 - accuracy: 0.9926 - val_loss: 0.0230 - val_acc

In [39]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

1772/1772 [==============================] - 2s 1ms/step
Accuracy: 99.32%
Precision: 1.00
Recall: 0.99
F1 Score: 1.00


In [41]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

In [49]:
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for RNN: (samples, time steps, features)
# Assuming each sample is a sequence of features (here, we treat each sample as 1 time step)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [51]:
model = Sequential()

# Add an RNN layer
model.add(SimpleRNN(units=100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# Add a Dense output layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 100)               11600     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,701
Trainable params: 11,701
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 14s 4ms/step - loss: 0.0329 - accuracy: 0.9914 - val_loss: 0.0221 - val_accuracy: 0.9928
Epoch 2/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0202 - accuracy: 0.9934 - val_loss: 0.0199 - val_accuracy: 0.9928
Epoch 3/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0184 - accuracy: 0.9935 - val_loss: 0.0185 - val_accuracy: 0.9928
Epoch 4/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0174 - accuracy: 0.9938 - val_loss: 0.0176 - val_accuracy: 0.9930
Epoch 5/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0166 - accuracy: 0.9939 - val_loss: 0.0174 - val_accuracy: 0.9932
Epoch 6/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0158 - accuracy: 0.9938 - val_loss: 0.0170 - val_accuracy: 0.9936
Epoch 7/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0152 - accuracy: 0.9940 - val_loss: 0.0158 - val_acc

In [61]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

1772/1772 [==============================] - 2s 1ms/step
Accuracy: 99.51%
Precision: 1.00
Recall: 1.00
F1 Score: 1.00


In [63]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [65]:
# Assume the last column is the label
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for LSTM: (samples, time steps, features)
# Assuming each sample has a sequence of features (here, each sample has 1 time step)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [67]:
model = Sequential()

# Add LSTM layer
model.add(LSTM(units=100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# Add Dense output layer for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               46400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 46,501
Trainable params: 46,501
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 15s 4ms/step - loss: 0.0398 - accuracy: 0.9902 - val_loss: 0.0229 - val_accuracy: 0.9928
Epoch 2/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0206 - accuracy: 0.9935 - val_loss: 0.0205 - val_accuracy: 0.9929
Epoch 3/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0186 - accuracy: 0.9936 - val_loss: 0.0190 - val_accuracy: 0.9930
Epoch 4/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0173 - accuracy: 0.9936 - val_loss: 0.0176 - val_accuracy: 0.9931
Epoch 5/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0163 - accuracy: 0.9938 - val_loss: 0.0173 - val_accuracy: 0.9935
Epoch 6/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0155 - accuracy: 0.9938 - val_loss: 0.0170 - val_accuracy: 0.9929
Epoch 7/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0150 - accuracy: 0.9940 - val_loss: 0.0159 - val_accurac

In [75]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

1772/1772 [==============================] - 1s 713us/step
Accuracy: 99.52%
Precision: 1.00
Recall: 1.00
F1 Score: 1.00


In [77]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [79]:
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [81]:
model = Sequential()

# Input layer and hidden layers
model.add(Dense(units=128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))

# Output layer (binary classification)
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               2048      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12,417
Trainable params: 12,417
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 6s 1ms/step - loss: 0.0258 - accuracy: 0.9926 - val_loss: 0.0191 - val_accuracy: 0.9929
Epoch 2/10
3307/3307 [==============================] - 4s 1ms/step - loss: 0.0178 - accuracy: 0.9937 - val_loss: 0.0166 - val_accuracy: 0.9934
Epoch 3/10
3307/3307 [==============================] - 4s 1ms/step - loss: 0.0155 - accuracy: 0.9939 - val_loss: 0.0155 - val_accuracy: 0.9936
Epoch 4/10
3307/3307 [==============================] - 5s 2ms/step - loss: 0.0124 - accuracy: 0.9953 - val_loss: 0.0152 - val_accuracy: 0.9953
Epoch 5/10
3307/3307 [==============================] - 4s 1ms/step - loss: 0.0099 - accuracy: 0.9964 - val_loss: 0.0118 - val_accuracy: 0.9960
Epoch 6/10
3307/3307 [==============================] - 4s 1ms/step - loss: 0.0082 - accuracy: 0.9967 - val_loss: 0.0071 - val_accuracy: 0.9973
Epoch 7/10
3307/3307 [==============================] - 5s 1ms/step - loss: 0.0075 - accuracy: 0.9971 - val_loss: 0.0090 - val_accuracy:

In [84]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

1772/1772 [==============================] - 2s 1ms/step
Accuracy: 99.80%
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
